# Combinando Datasets: Merge y Join

Una característica esencial que ofrece Pandas son sus operaciones de join y merge en memoria de alto rendimiento.
Si alguna vez has trabajado con bases de datos, deberías estar familiarizado con este tipo de interacción de datos.
La interfaz principal para esto es la función ``pd.merge``, y veremos algunos ejemplos de cómo puede funcionar en la práctica.

Por comodidad, empezaremos redefiniendo la función ``display()`` de la sección anterior:

In [ ]:
import pandas as pd
import numpy as np

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

## Álgebra relacional

El comportamiento implementado en ``pd.merge()`` es un subconjunto de lo que se conoce como *álgebra relacional*, que es un conjunto formal de reglas para la manipulación de datos relacionales, y forma la base conceptual de las operaciones disponibles en la mayoría de las bases de datos.
La fuerza del enfoque del álgebra relacional es que propone varias operaciones primitivas, que se convierten en los bloques de construcción de operaciones más complicadas en cualquier conjunto de datos.
Con este léxico de operaciones fundamentales implementado eficientemente en una base de datos u otro programa, se puede realizar una amplia gama de operaciones compuestas bastante complicadas.

Pandas implementa varios de estos bloques de construcción fundamentales en la función ``pd.merge()`` y el método relacionado ``join()`` de ``Series`` y ``Dataframe``.
Como veremos, estos permiten enlazar eficazmente datos de diferentes fuentes.

Traducción realizada con la versión gratuita del traductor www.DeepL.com/Translator

## Categorías de uniones

La función ``pd.merge()`` implementa varios tipos de joins: los joins *one-to-one*, *many-to-one*, y *many-to-many*.
Se accede a los tres tipos de joins mediante una llamada idéntica a la interfaz ``pd.merge()``; el tipo de join realizado depende de la forma de los datos de entrada.
Aquí mostraremos ejemplos sencillos de los tres tipos de merges, y discutiremos las opciones detalladas más adelante.

### Joins One-to-one

Quizás el tipo más sencillo de expresión de merge es el join one-to-one, que es en muchos aspectos muy similar a la concatenación por columnas vista en [Combinación de Datasets: Concat & Append](03.06-Concat-And-Append.ipynb).
Como ejemplo concreto, considere los siguientes dos ``DataFrames`` que contienen información sobre varios empleados de una empresa:

In [ ]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
display('df1', 'df2')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


Para combinar esta información en un único ``DataFrame``, podemos utilizar la función ``pd.merge()``:

In [ ]:
df3 = pd.merge(df1, df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


La función ``pd.merge()`` reconoce que cada ``DataFrame`` tiene una columna " employee", y la une automáticamente utilizando esta columna como clave.
El resultado de la merge es un nuevo ``DataFrame`` que combina la información de los dos inputs.
Observe que el orden de las entradas en cada columna no se mantiene necesariamente: en este caso, el orden de la columna " employee" difiere entre ``df1`` y ``df2``, y la función ``pd.merge()`` lo tiene en cuenta correctamente.
Además, hay que tener en cuenta que la merge en general descarta el índice, excepto en el caso especial de las merge por índice (ver las palabras clave ``left_index`` y ``right_index``, que se discutirán más adelante).

### Joins Many-to-one

Los joins many-to-one son joins en los que una de las dos columnas clave contiene entradas duplicadas.
En el caso de los joins múltiples, el ``DataFrame`` resultante conservará esas entradas duplicadas según corresponda.
Considere el siguiente ejemplo de un join many-to-one:

In [ ]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
display('df3', 'df4', 'pd.merge(df3, df4)')

df3
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

df4
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve

pd.merge(df3, df4)
  employee        group  hire_date supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012      Guido
2     Lisa  Engineering       2004      Guido
3      Sue           HR       2014      Steve

El ``DataFrame`` resultante tiene una columna adicional con la información del "supervisor", donde la información se repite en uno o más lugares según lo requieran los inputs.

### Joins Many-to-many 

Los joins many-to-many son un poco confusas conceptualmente, pero sin embargo están bien definidas.
Si la columna key en ambas arrays, izquierda y derecha, contiene duplicados, el resultado es una merge de muchos a muchos.
Esto quizá quede más claro con un ejemplo concreto.
Considere lo siguiente, donde tenemos un ``DataFrame`` que muestra una o más habilidades asociadas a un grupo particular.
Realizando un join many-to-many, podemos recuperar las habilidades asociadas a cualquier persona individual:

In [ ]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
display('df1', 'df5', "pd.merge(df1, df5)")

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets


Estos tres tipos de joins se pueden utilizar con otras herramientas de Pandas para implementar una amplia gama de funcionalidades.
Pero en la práctica, los datasets raramente son tan limpios como el que estamos trabajando aquí.
En la siguiente sección consideraremos algunas de las opciones proporcionadas por ``pd.merge()`` que permiten ajustar el funcionamiento de las operaciones join.

## Especificación de la Merge Key

Ya hemos visto el comportamiento por defecto de ``pd.merge()``: busca uno o más nombres de columna que coincidan entre los dos inputs, y lo utiliza como key.
Sin embargo, a menudo los nombres de las columnas no coinciden tan bien, y ``pd.merge()`` proporciona una variedad de opciones para manejar esto.

### La palabra clave ``on``

Lo más sencillo es especificar explícitamente el nombre de la columna clave mediante la palabra clave ``on``, que toma un nombre de columna o una lista de nombres de columna:

In [ ]:
display('df1', 'df2', "pd.merge(df1, df2, on='employee')")

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df2
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014

pd.merge(df1, df2, on='employee')
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

Esta opción sólo funciona si tanto el ``DataFrame`` izquierdo como el derecho tienen el nombre de columna especificado.

### Las keywords ``left_on`` y ``right_on``

A veces se desea mergear dos datasets con diferentes nombres de columna; por ejemplo, podemos tener un dataset en el que el nombre del empleado está etiquetado como " name " en lugar de " employee ".
En este caso, podemos utilizar las palabras clave ``left_on`` y ``right_on`` para especificar los dos nombres de columna:

In [ ]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
display('df1', 'df3', 'pd.merge(df1, df3, left_on="employee", right_on="name")')

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

pd.merge(df1, df3, left_on="employee", right_on="name")
  employee        group  name  salary
0      Bob   Accounting   Bob   70000
1     Jake  Engineering  Jake   80000
2     Lisa  Engineering  Lisa  120000
3      Sue           HR   Sue   90000

El resultado tiene una columna redundante que podemos eliminar si lo deseamos, por ejemplo, utilizando el método ``drop()`` de ``DataFrame``:

In [ ]:
pd.merge(df1, df3, left_on="employee", right_on="name").drop('name', axis=1)

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


### Las palabras clave ``left_index`` y ``right_index``

A veces, en lugar de mergear en una columna, se quiere mergear en un índice.
Por ejemplo, sus datos podrían tener este aspecto:

In [ ]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')
display('df1a', 'df2a')

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


Puedes utilizar el índice como clave para la fusión especificando los indicadores ``left_index`` y/o ``right_index`` en ``pd.merge()``:

In [ ]:
display('df1a', 'df2a',
        "pd.merge(df1a, df2a, left_index=True, right_index=True)")

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


Por comodidad, los ``DataFrame`` implementan el método ``join()``, que realiza un merge que por defecto une los índices:

In [ ]:
display('df1a', 'df2a', 'df1a.join(df2a)')

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


Si quieres mezclar índices y columnas, puedes combinar ``left_index`` con ``right_on`` o ``left_on`` con ``right_index`` para obtener el comportamiento deseado:

In [ ]:
display('df1a', 'df3', "pd.merge(df1a, df3, left_index=True, right_on='name')")

df1a
                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

pd.merge(df1a, df3, left_index=True, right_on='name')
         group  name  salary
0   Accounting   Bob   70000
1  Engineering  Jake   80000
2  Engineering  Lisa  120000
3           HR   Sue   90000

Todas estas opciones también funcionan con múltiples índices y/o múltiples columnas; la interfaz para este comportamiento es muy intuitiva.
Para más información sobre esto, consulte la sección ["Merge, Join, and Concatenate"](http://pandas.pydata.org/pandas-docs/stable/merging.html) de la documentación de Pandas.

## Especificación de la aritmética de conjuntos para Joins

En todos los ejemplos anteriores hemos pasado por alto una consideración importante a la hora de realizar una unión: el tipo de aritmética de conjuntos utilizado en la unión.
Esto surge cuando un valor aparece en una columna clave pero no en la otra. Considere este ejemplo:

In [ ]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']},
                   columns=['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                   columns=['name', 'drink'])
display('df6', 'df7', 'pd.merge(df6, df7)')

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7)
   name   food drink
0  Mary  bread  wine

Aquí hemos mergeado dos conjuntos de datos que sólo tienen una entrada de "name" en común: María.
Por defecto, el resultado contiene la *intersección* de los dos sets de inputs; esto es lo que se conoce como un *inner join*.
Podemos especificar esto explícitamente utilizando la palabra clave ``how``, que por defecto es ``inner``:

In [ ]:
pd.merge(df6, df7, how='inner')

,name,food,drink
0,Mary,bread,wine


Otras opciones para la keyword ``how`` son ``'outer'``, ``'left'``, y ``'right'``.
Un *outer join* devuelve unjoin sobre la unión de las columnas de inputs, y rellena todos los valores que faltan con NAs:

In [ ]:
display('df6', 'df7', "pd.merge(df6, df7, how='outer')")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='outer')
     name   food drink
0   Peter   fish   NaN
1    Paul  beans   NaN
2    Mary  bread  wine
3  Joseph    NaN  beer

El *left join* y el *right join* devuelven joins sobre las entradas de la izquierda y de la derecha, respectivamente.
Por ejemplo:

In [ ]:
display('df6', 'df7', "pd.merge(df6, df7, how='left')")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='left')
    name   food drink
0  Peter   fish   NaN
1   Paul  beans   NaN
2   Mary  bread  wine

Las filas de salida corresponden ahora a las entradas del input izquierdo. Utilizando ``how='right'`` funciona de forma similar.

Todas estas opciones pueden aplicarse directamente a cualquiera de los tipos de joins anteriores.

## Superposición de nombres de columnas: La keyword ``suffixes``

Por último, puede darse el caso de que sus dos inputs ``DataFrame`` tengan nombres de columna contradictorios.
Considere este ejemplo:

In [ ]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [1, 2, 3, 4]})
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [3, 1, 4, 2]})
display('df8', 'df9', 'pd.merge(df8, df9, on="name")')

df8
   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

df9
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

pd.merge(df8, df9, on="name")
   name  rank_x  rank_y
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2

Dado que la salida tendría dos nombres de columna conflictivos, la función merge añade automáticamente un sufijo ``_x`` o ``_y`` para que las columnas de salida sean únicas.
Si estos valores por defecto son inapropiados, es posible especificar un sufijo personalizado utilizando la palabra clave ``suffixes``:

In [ ]:
display('df8', 'df9', 'pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])')

df8
   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

df9
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])
   name  rank_L  rank_R
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2

Estos sufijos funcionan en cualquiera de los posibles patrones joins, y funcionan también si hay varias columnas superpuestas.

Para más información sobre estos patrones, vea [Agregación y agrupación](03.08-Agregación-y-agrupación.ipynb) donde nos sumergimos un poco más en el álgebra relacional.
También vea la documentación de Pandas "Merge, Join and Concatenate" (http://pandas.pydata.org/pandas-docs/stable/merging.html) para una mayor discusión de estos temas.

## Ejemplo: Datos de los Estados Unidos

Las operaciones merge y join aparecen con mayor frecuencia cuando se combinan datos de diferentes fuentes.
Aquí consideraremos un ejemplo de algunos datos sobre los estados de EE.UU. y su población.
Los archivos de datos se pueden encontrar en http://github.com/jakevdp/data-USstates/:

In [ ]:
# Following are shell commands to download the data
# !curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv
# !curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv
# !curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv

Echemos un vistazo a los tres conjuntos de datos, utilizando la función Pandas ``read_csv()``:

In [ ]:
pop = pd.read_csv('data/state-population.csv')
areas = pd.read_csv('data/state-areas.csv')
abbrevs = pd.read_csv('data/state-abbrevs.csv')

display('pop.head()', 'areas.head()', 'abbrevs.head()')

pop.head()
  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0

areas.head()
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707

abbrevs.head()
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA

Teniendo en cuenta esta información, digamos que queremos calcular un resultado relativamente sencillo: clasificar los estados y territorios de EE.UU. según su densidad de población en 2010.
Está claro que tenemos los datos para encontrar este resultado, pero tendremos que combinar los conjuntos de datos para encontrar el resultado.

Empezaremos con un merge many-to-one que nos dará el nombre completo del estado dentro del ``DataFrame`` de población.
Queremos mergear basándonos en la columna ``estado/región`` de ``pop``, y en la columna ``abreviación`` de ``abbrevs``.
Utilizaremos ``how='outer'`` para asegurarnos de que no se desecha ningún dato debido a la falta de coincidencia de las etiquetas.

In [ ]:
merged = pd.merge(pop, abbrevs, how='outer',
                  left_on='state/region', right_on='abbreviation')
merged = merged.drop('abbreviation', 1) # drop duplicate info
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


Comprobemos de nuevo si hay algún desajuste aquí, lo que podemos hacer buscando filas con nulos:

In [ ]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

Algunos datos de la ``population`` son nulos; ¡descubramos cuáles son!

In [ ]:
merged[merged['population'].isnull()].head()

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN


Parece que todos los valores de población nulos son de Puerto Rico anteriores al año 2000; esto se debe probablemente a que estos datos no están disponibles en la fuente original.

Y lo que es más importante, vemos también que algunas de las nuevas entradas de ``state`` son también nulas, lo que significa que no había ninguna entrada correspondiente en la key ``abbrevs``.
Averigüemos qué regiones carecen de esta coincidencia:

In [ ]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

Podemos deducir rápidamente el problema: nuestros datos de población incluyen entradas para Puerto Rico (PR) y los Estados Unidos en su conjunto (USA), mientras que estas entradas no aparecen en la clave de abreviatura del estado.
Podemos arreglar esto rápidamente rellenando las entradas apropiadas:

In [ ]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

Se acabaron los nulos en la columna ``state``: ¡ya está todo listo!

Ahora podemos mergear el resultado con los datos del área utilizando un procedimiento similar.
Examinando nuestros resultados, querremos joins en la columna ``state`` en ambos:

In [ ]:
final = pd.merge(merged, areas, on='state', how='left')
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


De nuevo, vamos a comprobar los nulos para ver si hay algún desajuste:

In [ ]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

Hay nulos en la columna ``área``; podemos echar un vistazo para ver qué regiones fueron ignoradas aquí:

In [ ]:
final['state'][final['area (sq. mi)'].isnull()].unique()

array(['United States'], dtype=object)

Vemos que nuestro ``areas`` ``DataFrame`` no contiene el área de los Estados Unidos en su conjunto.
Podríamos insertar el valor apropiado (utilizando la suma de las áreas de todos los estados, por ejemplo), pero en este caso simplemente dejaremos los valores nulos porque la densidad de población de todo Estados Unidos no es relevante para nuestra discusión actual:

In [ ]:
final.dropna(inplace=True)
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


Ahora tenemos todos los datos que necesitamos. Para responder a la pregunta que nos interesa, vamos a seleccionar primero la parte de los datos correspondiente al año 2000, y la población total.
Utilizaremos la función ``query()`` para hacerlo rápidamente (para ello es necesario tener instalado el paquete ``numexpr``; ver [High-Performance Pandas: ``eval()`` y ``query()``](03.12-Performance-Eval-and-Query.ipynb)):

In [ ]:
data2010 = final.query("year == 2010 & ages == 'total'")
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


Ahora vamos a calcular la densidad de población y a mostrarla en orden.
Empezaremos por volver a indexar nuestros datos en el estado y luego calcularemos el resultado:

In [ ]:
data2010.set_index('state', inplace=True)
density = data2010['population'] / data2010['area (sq. mi)']

In [ ]:
density.sort_values(ascending=False, inplace=True)
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

El resultado es una clasificación de los estados de EE.UU. más Washington, DC, y Puerto Rico por orden de su densidad de población en 2010, en residentes por milla cuadrada.
Podemos ver que, con diferencia, la región más densa en este conjunto de datos es Washington DC (es decir, el Distrito de Columbia); entre los estados, el más denso es Nueva Jersey.

También podemos comprobar el final de la lista:

In [ ]:
density.tail()

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64

Vemos que el estado menos denso, con diferencia, es Alaska, con una media de poco más de un residente por milla cuadrada.

Este tipo de messy data merging es una tarea común cuando se trata de responder a preguntas utilizando fuentes de datos del mundo real.
Espero que este ejemplo te haya dado una idea de las formas en que puedes combinar las herramientas que hemos cubierto para obtener información de tus datos.